In [1]:
import pandas as pd
import geopandas as gpd
import plotly
import plotly.graph_objects as gp

In [2]:
df_b400 = gpd.read_file("../DatosProcesados/AreaInfluencia/predio_buffer_400m.shp")
df_b1200 = gpd.read_file("../DatosProcesados/AreaInfluencia/predio_buffer_1200m.shp")
df_ttim10min_b1200 = gpd.read_file("../DatosProcesados/AreaInfluencia/predio_ttime10min_1200m_manual.shp")
df_CiudadEvita = gpd.read_file("../QGIS_projects/Datos/AreaGeográfica/CiudadEvita.shp")
df_IsidroCasanova = gpd.read_file("../QGIS_projects/Datos/AreaGeográfica/IsidroCasanova.shp")
df_muni = gpd.read_file("D:/JOB/UNSAM/2022 - CII/Datos/municipio/municipio.shp")
df_muni = df_muni.loc[df_muni.nam == 'La Matanza', ['geometry']]

In [3]:
# census data
df_census = gpd.read_file("../DatosProcesados/Sociodemograficos/DatosCensales.geojson", driver='GeoJSON')

In [4]:
df_agesex = pd.read_csv("../DatosProcesados/redatam/LM_pob_sexage.csv")
df_agesex['link'] = "0"+ df_agesex['radio_link'].astype(str)
df_agesex = df_agesex.loc[df_agesex.link != '064279724']
df_agesex.head()

,Unnamed: 0,radio_link,range,varon,mujer,total,link
0,0,64270101,0-4,14,5,19,064270101
1,1,64270101,5-9,14,9,23,064270101
2,2,64270101,10-14,5,7,12,064270101
3,3,64270101,15-19,7,7,14,064270101
4,4,64270101,20-24,6,11,17,064270101


In [5]:
df_agesex_CABA = pd.read_csv("../Datos/redatam/poblacion_por_sexo_y_edad_CABA.csv", encoding='latin')
df_agesex_CABA.varon = df_agesex_CABA.varon * 1000
df_agesex_CABA.mujer = df_agesex_CABA.mujer * 1000
df_agesex_CABA.total = df_agesex_CABA.total * 1000
df_agesex_CABA.head(20)

,range,varon,mujer,total
0,0-4,84382.0,81256.0,165638.0
1,5-9,79472.0,76900.0,156372.0
2,10-14,76354.0,74147.0,150501.0
3,15-19,83338.0,84343.0,167681.0
4,20-24,110915.0,117210.0,228125.0
5,25-29,120567.0,127027.0,247594.0
6,30-34,120215.0,127854.0,248069.0
7,35-39,103249.0,112077.0,215326.0
8,40-44,85451.0,95425.0,180876.0
9,45-49,79456.0,92170.0,171626.0


# Ploteo por área

In [7]:
for i in [[df_b400, 'Área 400m'], [df_b1200, 'Área 1200m'], [df_ttim10min_b1200, 'Área Interés'], [df_CiudadEvita, 'Ciudad Evita'], [df_IsidroCasanova, 'Isidro Casanova'], [df_muni, 'La Matanza']]:
    df_area = i[0]
    area_geo = i[1]

    df_data_radios = df_census.sjoin(df_area[['geometry']])
    df_data_radios_data = df_data_radios[['link']].merge(df_agesex, on='link')
    df_data_radios_data= df_data_radios_data.groupby(['range'])[['varon', 'mujer']].sum().reset_index()
    
    df_data_radios_data['range_order'] = df_data_radios_data.apply(lambda row: row['range'].split("-")[0], axis=1)
    df_data_radios_data.loc[df_data_radios_data.range_order == '95 y más', 'range_order'] = 95
    df_data_radios_data.range_order = df_data_radios_data.range_order.astype(int)
    df_data_radios_data = df_data_radios_data.sort_values('range_order').reset_index(drop=True)

    y_age = df_data_radios_data['range']
    x_M = df_data_radios_data['varon']
    x_F = df_data_radios_data['mujer'] * -1

    # Creating instance of the figure
    fig = gp.Figure()
    
    # Adding Male data to the figure
    fig.add_trace(gp.Bar(y= y_age, 
                        x = x_M, 
                        name = 'Varones', 
                        orientation = 'h',
                        text=x_M,
                        marker_color='#5B7DB3'))

    # Adding Female data to the figure
    fig.add_trace(gp.Bar(y = y_age, 
                        x = x_F,
                        name = 'Mujeres', 
                        orientation = 'h',
                        text=x_F*(-1),
                        marker_color='#E19393'))
    
    # Updating the layout for our graph
    fig.update_layout(title = 'Piramide Poblacional: {}'.format(area_geo),
                    title_font_size = 22, 
                    width=700,
                    height=700,
                    # paper_bgcolor='rgba(0,0,0,0)',
                    plot_bgcolor='rgba(0,0,0,0)',
                    barmode = 'relative',
                    bargap = 0.0, 
                    bargroupgap = 0,
                    font=dict(
                        family="Calibri",
                        size=14,
                        color="darkgrey"
                    ),
                    xaxis = dict(
                        showticklabels= False,
                        title = 'Población',
                        title_font_size = 14)
                    )
                    
    fig.write_image("../plots/piramide_{}.png".format(area_geo))
    fig.write_image("../plots/piramide_{}.svg".format(area_geo))
    fig.write_html("../plots/piramide_{}.html".format(area_geo))
    fig.show()

In [8]:
# 'Área Interés'

df_ttim10min_b1200_radios = df_census.sjoin(df_ttim10min_b1200[['geometry']])
df_ttim10min_b1200_radios_data = df_ttim10min_b1200_radios[['link']].merge(df_agesex, on='link')
df_ttim10min_b1200_radios_data= df_ttim10min_b1200_radios_data.groupby(['range'])[['varon', 'mujer']].sum().reset_index()

df_ttim10min_b1200_radios_data['range_order'] = df_ttim10min_b1200_radios_data.apply(lambda row: row['range'].split("-")[0], axis=1)
df_ttim10min_b1200_radios_data.loc[df_ttim10min_b1200_radios_data.range_order == '95 y más', 'range_order'] = 95
df_ttim10min_b1200_radios_data.range_order = df_ttim10min_b1200_radios_data.range_order.astype(int)
df_ttim10min_b1200_radios_data = df_ttim10min_b1200_radios_data.sort_values('range_order').reset_index(drop=True)

y_age = df_ttim10min_b1200_radios_data['range']
x_M_AreaInteres = df_ttim10min_b1200_radios_data['varon']
x_F_AreaInteres = df_ttim10min_b1200_radios_data['mujer'] * -1

# 'La Matanza'

df_ttim10min_b1200_radios = df_census.sjoin(df_muni[['geometry']])
df_ttim10min_b1200_radios_data = df_ttim10min_b1200_radios[['link']].merge(df_agesex, on='link')
df_ttim10min_b1200_radios_data= df_ttim10min_b1200_radios_data.groupby(['range'])[['varon', 'mujer']].sum().reset_index()

df_ttim10min_b1200_radios_data['range_order'] = df_ttim10min_b1200_radios_data.apply(lambda row: row['range'].split("-")[0], axis=1)
df_ttim10min_b1200_radios_data.loc[df_ttim10min_b1200_radios_data.range_order == '95 y más', 'range_order'] = 95
df_ttim10min_b1200_radios_data.range_order = df_ttim10min_b1200_radios_data.range_order.astype(int)
df_ttim10min_b1200_radios_data = df_ttim10min_b1200_radios_data.sort_values('range_order').reset_index(drop=True)

y_age = df_ttim10min_b1200_radios_data['range']
x_M_LaMatanza = df_ttim10min_b1200_radios_data['varon']
x_F_LaMatanza = df_ttim10min_b1200_radios_data['mujer'] * -1

In [9]:
# Creating instance of the figure
fig = gp.Figure()

# Adding Male data to the figure
fig.add_trace(gp.Bar(y= y_age, 
                    x = x_M_AreaInteres, 
                    name = 'Varones en Área de Interés', 
                    orientation = 'h',
                    text= round((100*x_M_AreaInteres/x_M_LaMatanza), 1)[:10],
                    marker_color='#5B7DB3'))
fig.add_trace(gp.Bar(y= y_age, 
                    x = x_M_LaMatanza-x_M_AreaInteres, 
                    name = 'Varones en {}'.format("La Matanza"), 
                    text= x_M_LaMatanza,
                    orientation = 'h',
                    marker_color='#A1B4D3'))


# Adding Female data to the figure
fig.add_trace(gp.Bar(y = y_age, 
                    x = x_F_AreaInteres,
                    name = 'Mujeres en Área de Interés', 
                    text= round((100*x_F_AreaInteres/x_F_LaMatanza), 1)[:10],
                    orientation = 'h',
                    marker_color='#E19393'))
fig.add_trace(gp.Bar(y = y_age, 
                    x = x_F_LaMatanza-x_F_AreaInteres,
                    name = 'Mujeres en {}'.format("La Matanza"), 
                    text=(x_F_LaMatanza)*-1,
                    orientation = 'h',
                    marker_color='#ECBBBB'))
                    
# Updating the layout for our graph
fig.update_layout(title = 'Piramide Poblacional: {}'.format("Área de Interés en La Matanza"),
                title_font_size = 22, 
                width=1200,
                height=700,
                # paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                barmode = 'relative',
                bargap = 0.0, 
                bargroupgap = 0,
                font=dict(
                    family="Calibri",
                    size=14,
                    color="darkgrey"
                ),
                xaxis = dict(
                    showticklabels= False,
                    title = 'Población',
                    title_font_size = 14)
                )
                
fig.write_image("../plots/piramide_{}.png".format("AreaInteresLaMatanza"))
fig.write_image("../plots/piramide_{}.svg".format("AreaInteresLaMatanza"))
fig.write_html("../plots/piramide_{}.html".format("AreaInteresLaMatanza"))
fig.show()

In [10]:
y_age = df_agesex_CABA['range']
x_M_CABA = (df_agesex_CABA['varon']).astype(int)
x_F_CABA = (df_agesex_CABA['mujer'] * -1 ).astype(int)

# Creating instance of the figure
fig = gp.Figure()

# Adding Male data to the figure
fig.add_trace(gp.Bar(y= y_age, 
                    x = x_M_AreaInteres, 
                    name = 'Varones en Área de Interés', 
                    orientation = 'h',
                    text= round((100*x_M_AreaInteres/x_M_CABA), 1)[:10],
                    marker_color='#5B7DB3'))
fig.add_trace(gp.Bar(y= y_age, 
                    x = x_M_CABA-x_M_AreaInteres, 
                    name = 'Varones en {}'.format("CABA"), 
                    text= x_M_CABA,
                    orientation = 'h',
                    marker_color='#A1B4D3'))


# Adding Female data to the figure
fig.add_trace(gp.Bar(y = y_age, 
                    x = x_F_AreaInteres,
                    name = 'Mujeres en Área de Interés', 
                    text= round((100*x_F_AreaInteres/x_F_CABA), 1)[:10],
                    orientation = 'h',
                    marker_color='#E19393'))
fig.add_trace(gp.Bar(y = y_age, 
                    x = x_F_CABA-x_F_AreaInteres,
                    name = 'Mujeres en {}'.format("CABA"), 
                    text=(x_F_CABA)*-1,
                    orientation = 'h',
                    marker_color='#ECBBBB'))
                    
# Updating the layout for our graph
fig.update_layout(title = 'Piramide Poblacional: {}'.format("Área de Interés respecto a CABA"),
                title_font_size = 22, 
                width=1200,
                height=700,
                # paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                barmode = 'relative',
                bargap = 0.0, 
                bargroupgap = 0,
                font=dict(
                    family="Calibri",
                    size=14,
                    color="darkgrey"
                ),
                xaxis = dict(
                    showticklabels= False,
                    title = 'Población',
                    title_font_size = 14)
                )
                
fig.write_image("../plots/piramide_{}.png".format("AreaInteresCABA"))
fig.write_image("../plots/piramide_{}.svg".format("AreaInteresCABA"))
fig.write_html("../plots/piramide_{}.html".format("AreaInteresCABA"))
fig.show()

In [11]:
# 'Ciudad Evita'

df_ttim10min_b1200_radios = df_census.sjoin(df_CiudadEvita[['geometry']])
df_ttim10min_b1200_radios_data = df_ttim10min_b1200_radios[['link']].merge(df_agesex, on='link')
df_ttim10min_b1200_radios_data= df_ttim10min_b1200_radios_data.groupby(['range'])[['varon', 'mujer']].sum().reset_index()

df_ttim10min_b1200_radios_data['range_order'] = df_ttim10min_b1200_radios_data.apply(lambda row: row['range'].split("-")[0], axis=1)
df_ttim10min_b1200_radios_data.loc[df_ttim10min_b1200_radios_data.range_order == '95 y más', 'range_order'] = 95
df_ttim10min_b1200_radios_data.range_order = df_ttim10min_b1200_radios_data.range_order.astype(int)
df_ttim10min_b1200_radios_data = df_ttim10min_b1200_radios_data.sort_values('range_order').reset_index(drop=True)

y_age = df_ttim10min_b1200_radios_data['range']
x_M_CiudadEvita = df_ttim10min_b1200_radios_data['varon']
x_F_CiudadEvita = df_ttim10min_b1200_radios_data['mujer'] * -1

# 'Isidro Casanova'

df_ttim10min_b1200_radios = df_census.sjoin(df_IsidroCasanova[['geometry']])
df_ttim10min_b1200_radios_data = df_ttim10min_b1200_radios[['link']].merge(df_agesex, on='link')
df_ttim10min_b1200_radios_data= df_ttim10min_b1200_radios_data.groupby(['range'])[['varon', 'mujer']].sum().reset_index()

df_ttim10min_b1200_radios_data['range_order'] = df_ttim10min_b1200_radios_data.apply(lambda row: row['range'].split("-")[0], axis=1)
df_ttim10min_b1200_radios_data.loc[df_ttim10min_b1200_radios_data.range_order == '95 y más', 'range_order'] = 95
df_ttim10min_b1200_radios_data.range_order = df_ttim10min_b1200_radios_data.range_order.astype(int)
df_ttim10min_b1200_radios_data = df_ttim10min_b1200_radios_data.sort_values('range_order').reset_index(drop=True)

y_age = df_ttim10min_b1200_radios_data['range']
x_M_IsidroCasanova = df_ttim10min_b1200_radios_data['varon']
x_F_IsidroCasanova = df_ttim10min_b1200_radios_data['mujer'] * -1


In [12]:
# Creating instance of the figure
fig = gp.Figure()

# Adding Male data to the figure
fig.add_trace(gp.Bar(y= y_age, 
                    x = x_M_AreaInteres, 
                    name = 'Varones en Área de Interés', 
                    orientation = 'h',
                    text= x_M_AreaInteres, # round((100*x_M_AreaInteres/x_M_IsidroCasanova), 1)[:10],
                    marker_color='#5B7DB3'))
fig.add_trace(gp.Bar(y= y_age, 
                    x = x_M_IsidroCasanova-x_M_AreaInteres, 
                    name = 'Varones en {}'.format("Isidro Casanova"), 
                    text= x_M_IsidroCasanova,
                    orientation = 'h',
                    marker_color='#A1B4D3'))


# Adding Female data to the figure
fig.add_trace(gp.Bar(y = y_age, 
                    x = x_F_AreaInteres,
                    name = 'Mujeres en Área de Interés', 
                    text= x_F_AreaInteres*-1,# round((100*x_F_AreaInteres/x_F_IsidroCasanova), 1)[:10],
                    orientation = 'h',
                    marker_color='#E19393'))
fig.add_trace(gp.Bar(y = y_age, 
                    x = x_F_IsidroCasanova-x_F_AreaInteres,
                    name = 'Mujeres en {}'.format("Isidro Casanova"), 
                    text=(x_F_IsidroCasanova)*-1,
                    orientation = 'h',
                    marker_color='#ECBBBB'))
                    
# Updating the layout for our graph
fig.update_layout(title = 'Piramide Poblacional: {}'.format("Área de Interés respecto a Isidro Casanova"),
                title_font_size = 22, 
                width=1200,
                height=700,
                # paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                barmode = 'relative',
                bargap = 0.0, 
                bargroupgap = 0,
                font=dict(
                    family="Calibri",
                    size=14,
                    color="darkgrey"
                ),
                xaxis = dict(
                    showticklabels= False,
                    title = 'Población',
                    title_font_size = 14)
                )
                
fig.write_image("../plots/piramide_{}.png".format("AreaInteresIsidroCasanova"))
fig.write_image("../plots/piramide_{}.svg".format("AreaInteresIsidroCasanova"))
fig.write_html("../plots/piramide_{}.html".format("AreaInteresIsidroCasanova"))
fig.show()

In [13]:
# Creating instance of the figure
fig = gp.Figure()

# Adding Male data to the figure
fig.add_trace(gp.Bar(y= y_age, 
                    x = x_M_CiudadEvita, 
                    name = 'Varones en {}'.format("Ciudad Evita"), 
                    text= x_M_CiudadEvita,
                    orientation = 'h',
                    marker_color='#A1B4D3'))
fig.add_trace(gp.Bar(y= y_age, 
                    x = x_M_AreaInteres-x_M_CiudadEvita, 
                    name = 'Varones en Área de Interés', 
                    orientation = 'h',
                    text= x_M_AreaInteres, # round((100*x_M_AreaInteres/x_M_IsidroCasanova), 1)[:10],
                    marker_color='#5B7DB3'))



# Adding Female data to the figure
fig.add_trace(gp.Bar(y = y_age, 
                    x = x_F_CiudadEvita,
                    name = 'Mujeres en {}'.format("Ciudad Evita"), 
                    text=(x_F_CiudadEvita)*-1,
                    orientation = 'h',
                    marker_color='#ECBBBB'))
fig.add_trace(gp.Bar(y = y_age, 
                    x = x_F_AreaInteres-x_F_CiudadEvita,
                    name = 'Mujeres en Área de Interés', 
                    text= x_F_AreaInteres*-1,# round((100*x_F_AreaInteres/x_F_IsidroCasanova), 1)[:10],
                    orientation = 'h',
                    marker_color='#E19393'))

                    
# Updating the layout for our graph
fig.update_layout(title = 'Piramide Poblacional: {}'.format("Área de Interés respecto a Ciudad Evita"),
                title_font_size = 22, 
                width=1200,
                height=700,
                # paper_bgcolor='rgba(0,0,0,0)',
                plot_bgcolor='rgba(0,0,0,0)',
                barmode = 'relative',
                bargap = 0.0, 
                bargroupgap = 0,
                font=dict(
                    family="Calibri",
                    size=14,
                    color="darkgrey"
                ),
                xaxis = dict(
                    showticklabels= False,
                    title = 'Población',
                    title_font_size = 14)
                )
                
fig.write_image("../plots/piramide_{}.png".format("AreaInteresCiudadEvita"))
fig.write_image("../plots/piramide_{}.svg".format("AreaInteresCiudadEvita"))
fig.write_html("../plots/piramide_{}.html".format("AreaInteresCiudadEvita"))
fig.show()

In [16]:
fig = gp.Figure(data=[
    gp.Bar(name='Área de Interés', x=y_age, y= x_M_AreaInteres, marker_color="#496A9F"), # 5B7DB3
    gp.Bar(name='Ciudad Evita', x=y_age, y= x_M_CiudadEvita, marker_color="#7E99C3"),
    gp.Bar(name='Isidro Casanova', x=y_age, y= x_M_IsidroCasanova, marker_color="#A1B4D3")
])
# Change the bar mode  
fig.update_layout(
    title = '{}'.format("Cantidad total de varones"),
    title_font_size = 22,
    width=1200,
    height=500,
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    font=dict(
        family="Calibri",
        size=14,
        color="darkgrey"
    ),
    xaxis= dict(
        showline=True, linewidth=1, linecolor='darkgrey'
    ),
    yaxis= dict(
        gridwidth=0.5, gridcolor='lightgrey'
    )
    
)
fig.write_image("../plots/barrasparalelas_{}.png".format("Cantidad de Varones"))
fig.write_image("../plots/barrasparalelas_{}.svg".format("Cantidad de Varones"))
fig.write_html("../plots/barrasparalelas_{}.html".format("Cantidad de Varones"))
fig.show()

In [17]:
fig = gp.Figure(data=[
    gp.Bar(name='Área de Interés', x=y_age, y= x_F_AreaInteres*-1, marker_color="#D66969"), # E19393
    gp.Bar(name='Ciudad Evita', x=y_age, y= x_F_CiudadEvita*-1, marker_color="#E7A7A7"),
    gp.Bar(name='Isidro Casanova', x=y_age, y= x_F_IsidroCasanova*-1, marker_color="#ECBBBB")
])
# Change the bar mode  E7A7A7
fig.update_layout(
    title = '{}'.format("Cantidad total de mujeres"),
    title_font_size = 22,
    width=1200,
    height=500,
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    font=dict(
        family="Calibri",
        size=14,
        color="darkgrey"
    ),
    xaxis= dict(
        showline=True, linewidth=1, linecolor='darkgrey'
    ),
    yaxis= dict(
        gridwidth=0.5, gridcolor='lightgrey'
    )
    
)
fig.write_image("../plots/barrasparalelas_{}.png".format("Cantidad de Mujeres"))
fig.write_image("../plots/barrasparalelas_{}.svg".format("Cantidad de Mujeres"))
fig.write_html("../plots/barrasparalelas_{}.html".format("Cantidad de Mujeres"))
fig.show()